In [1]:
import pandas as pd
# to show all columns
pd.set_option('display.max_columns', None)


In [2]:
pd.read_parquet("interventions1.parquet") 

,Mission ID,Service Name,PostalCode permanence,CityName permanence,StreetName permanence,HouseNumber permanence,Latitude permanence,Longitude permanence,Permanence short name,Permanence long name,Vector type,EventType Firstcall,EventLevel Firstcall,EventType Trip,EventLevel Trip,PostalCode intervention,CityName intervention,Latitude intervention,Longitude intervention,Province intervention,T0,T1,T1confirmed,T2,T3,T4,T5,T6,T7,T9,Intervention time (T1Reported),Intervention time (T1Confirmed),Waiting time,Intervention duration,Departure time (T1Reported),Departure time (T1Confirmed),Unavailable time,Name destination hospital,PostalCode destination hospital,CityName destination hospital,StreetName destination hospital,HouseNumber destination hospital,Calculated travelTime destinatio,Calculated Distance destination,Number of transported persons,Abandon reason
0,10221520001,HA UR MECH AZ St Maarten,2800.0,Mechelen (Mechelen),Liersesteenweg,None,51.051020,4.478030,AAMECH01A,ZW MECHELEN 1,Ambulance,P020 - Intoxication alcohol,N5,P020 - Intoxication alcohol,N5,None,Lier (Lier),51.12496,4.57408,ANT,01JUN22:00:01:34,01JUN22:00:03:26,2022-06-01 00:20:14.417,2022-06-01 00:07:34.655,2022-06-01 00:17:53.888,None,None,2022-06-01 00:28:18.934,2022-06-01 00:53:17.876,None,14.0,0.0,16.0,27.0,4.0,0.0,NaN,None,None,None,None,None,NaN,NaN,NaN,Verzorgd ter plaatse
1,10221520002,BA KAPE AMBUCE,2950.0,Kapellen (Kapellen),Essenhoutstraat,None,51.312075,4.424398,AAKAPE01A,ZW KAPELLEN 1,Ambulance,P068 - Urogenital problem,N5,P068 - Urogenital problem,N5,None,Stabroek (Hoevenen),51.30626,4.40502,ANT,01JUN22:00:03:51,01JUN22:00:06:14,2022-06-01 00:06:19.101,2022-06-01 00:10:00.017,2022-06-01 00:16:25.356,2022-06-01 00:46:34.089,2022-06-01 00:56:48.871,2022-06-01 01:08:30.111,2022-06-01 01:25:44.611,None,10.0,10.0,13.0,65.0,4.0,4.0,NaN,HA UR ANTW Jan Palfijn,None,Antwerpen (Merksem),Lange Bremstraat,None,678.0,11743.0,1.0,None
2,10221520002,HA UR ANTW Stuivenberg,2060.0,Antwerpen (Antwerpen),Lange Beeldekensstraat,None,51.222490,4.436290,UAANTW01A,MUG ANTWERPEN 1,MUG,P068 - Urogenital problem,N5,P068 - Urogenital problem,N1,None,Stabroek (Hoevenen),51.30626,4.40502,ANT,01JUN22:00:03:51,01JUN22:00:22:18,2022-06-01 00:22:29.639,2022-06-01 00:25:35.404,2022-06-01 00:35:58.397,2022-06-01 00:46:30.026,2022-06-01 01:06:50.953,2022-06-01 01:06:52.142,2022-06-01 01:20:48.471,None,13.0,13.0,32.0,63.0,3.0,3.0,NaN,None,None,None,None,None,NaN,NaN,NaN,None
3,10221520004,BA ANTW [Borgerhout] AMBUCE,2140.0,Antwerpen (Borgerhout),Gijselsstraat,None,51.215620,4.443920,AAANTW07A,ZW ANTWERPEN 7,Ambulance,P033 - Trauma,N5,P033 - Trauma,N5,None,Antwerpen (Merksem),51.23878,4.44533,ANT,01JUN22:00:08:56,01JUN22:00:12:59,2022-06-01 00:15:21.088,2022-06-01 00:15:36.856,2022-06-01 00:27:35.760,None,None,2022-06-01 00:40:57.191,2022-06-01 00:48:03.555,None,15.0,12.0,19.0,32.0,3.0,0.0,NaN,None,None,None,None,None,NaN,NaN,NaN,Zonder patient
4,10221520005,BA WIJN AMBUCE,2110.0,Wijnegem (Wijnegem),Bijkhoevelaan,None,51.233550,4.493180,AAWIJN01A,ZW WIJNEGEM 1,Ambulance,P022 - Intoxication medication,N5,P022 - Intoxication medication,N5,None,Schoten (Schoten),51.25071,4.51299,ANT,01JUN22:00:10:38,01JUN22:00:15:00,2022-06-01 00:16:21.827,2022-06-01 00:18:01.345,2022-06-01 00:25:28.053,2022-06-01 00:50:17.593,2022-06-01 00:59:50.136,2022-06-01 01:23:34.290,None,None,10.0,9.0,15.0,73.0,3.0,2.0,NaN,HA UR ANTW Jan Palfijn,None,Antwerpen (Merksem),Lange Bremstraat,None,530.0,5468.0,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200622,50221920082,FH PDS BEAU Hainaut Est,6500.0,Beaumont (Beaumont),Chauss√©e de Mons,None,50.241870,4.234680,AHBEAU01A,ZS BEAUMONT 1,Ambulance,P033 - Trauma,N5,P033 - Trauma,N5,None,Sivry-Rance (Sivry),50.16330,4.17897,HAI,11JUL22:06:59:08,11JUL22:07:01:30,2022-07-11 07:01:43.273,2022-07-11 07:02:55.511,2022-07-11 07:13:

In [3]:
interventions1 = pd.read_parquet("interventions1.parquet") 


interventions1_car = interventions1[interventions1['EventType Trip'] == 'P003 - Cardiac arrest']
# keep only Latitude intervention, Longitude intervention, T0, T2, T3
interventions1_car = interventions1_car[['Latitude intervention', 'Longitude intervention', 'CityName permanence','T0', 'T2', 'T3']]
# calculate the T3-T0, and T2-T0 using datetime
interventions1_car['T2'] = pd.to_datetime(interventions1_car['T2']).dt.floor('S')
interventions1_car['T3'] = pd.to_datetime(interventions1_car['T3']).dt.floor('S')
interventions1_car['T0'] = pd.to_datetime(interventions1_car['T0'], format="%d%b%y:%H:%M:%S").dt.floor('S')
# calculate the time difference with minutes
interventions1_car['T3-T0'] = (interventions1_car['T3'] - interventions1_car['T0']).dt.total_seconds() / 60
interventions1_car['T2-T0'] = (interventions1_car['T2'] - interventions1_car['T0']).dt.total_seconds() / 60
interventions1_car['T3-T2'] = (interventions1_car['T3'] - interventions1_car['T2']).dt.total_seconds() / 60


# change the name of lat and long to Lat and Lon
interventions1_car.rename(columns={'Latitude intervention': 'Lat', 'Longitude intervention': 'Lon', 'CityName permanence': 'City'}, inplace=True)
interventions1_car['City'] = interventions1_car['City'].str.lower()

interventions1_car

,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
42,51.20685,5.13662,geel (geel),2022-06-01 06:45:53,2022-06-01 06:52:06,2022-06-01 07:04:46,18.883333,6.216667,12.666667
87,51.24848,4.43869,antwerpen (albertdok),2022-06-01 09:28:15,2022-06-01 09:30:28,2022-06-01 09:32:29,4.233333,2.216667,2.016667
88,51.24848,4.43869,antwerpen (antwerpen),2022-06-01 09:28:15,2022-06-01 09:35:10,2022-06-01 09:40:51,12.600000,6.916667,5.683333
207,51.28273,4.42134,antwerpen (antwerpen),2022-06-01 15:11:15,2022-06-01 15:14:47,2022-06-01 15:24:40,13.416667,3.533333,9.883333
208,51.28273,4.42134,antwerpen (merksem),2022-06-01 15:11:15,2022-06-01 15:17:56,2022-06-01 15:47:52,36.616667,6.683333,29.933333
...,...,...,...,...,...,...,...,...,...
199959,50.46763,4.10432,la louvi√®re (str√©py-bracquegnies),2022-07-09 17:33:17,2022-07-09 17:36:21,2022-07-09 17:38:13,4.933333,3.066667,1.866667
199960,50.46763,4.10432,mons (mons),2022-07-09 17:33:17,2022-07-09 17:40:02,2022-07-09 17:50:39,17.366667,6.750000,10.616667
200261,50.03854,4.26365,chimay (chimay),2022-07-10 11:21:00,2022-07-10 11:23:34,2022-07-10 11:28:33,7.550000,2.566667,4.983333
200388,50.53594,4.24645,nivelles (nivelles),2022-07-10 16:06:47,2022-07-10 16:11:18,2022-07-10 16:21:48,15.016667,4.516667,10.500000


In [4]:
interventions2 = pd.read_parquet("interventions2.parquet") 

interventions2_car = interventions2[interventions2['EventType Trip'] == 'P003 - Cardiac arrest']
interventions2_car = interventions2_car[['Latitude intervention', 'Longitude intervention', 'CityName permanence', 'T0', 'T2', 'T3']]
interventions2_car['T2'] = pd.to_datetime(interventions2_car['T2'], format="%Y-%m-%d %H:%M:%S.%f").dt.floor('S')
interventions2_car['T3'] = pd.to_datetime(interventions2_car['T3'], format="%Y-%m-%d %H:%M:%S.%f").dt.floor('S')
interventions2_car['T0'] = pd.to_datetime(interventions2_car['T0'], format="%d%b%y:%H:%M:%S").dt.floor('S')

interventions2_car['T3-T0'] = (interventions2_car['T3'] - interventions2_car['T0']).dt.total_seconds() / 60
interventions2_car['T2-T0'] = (interventions2_car['T2'] - interventions2_car['T0']).dt.total_seconds() / 60
interventions2_car['T3-T2'] = (interventions2_car['T3'] - interventions2_car['T2']).dt.total_seconds() / 60

interventions2_car.rename(columns={'Latitude intervention': 'Lat', 'Longitude intervention': 'Lon', 'CityName permanence': 'City'}, inplace=True)
interventions2_car['City'] = interventions2_car['City'].str.lower()

interventions2_car


,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
177,50.69656,4.03820,enghien (enghien),2022-07-11 15:26:25,2022-07-11 15:29:00,2022-07-11 15:30:51,4.433333,2.583333,1.850000
178,50.69656,4.03820,soignies (soignies),2022-07-11 15:26:25,2022-07-11 15:35:35,2022-07-11 16:18:51,52.433333,9.166667,43.266667
298,50.34275,4.47152,charleroi (marcinelle),2022-07-11 20:24:17,2022-07-11 20:30:07,2022-07-11 20:38:11,13.900000,5.833333,8.066667
362,50.59626,4.15701,braine-le-comte (braine-le-comte),2022-07-11 23:04:40,2022-07-11 23:08:28,2022-07-11 23:16:38,11.966667,3.800000,8.166667
427,50.40064,4.39426,charleroi (monceau-sur-sambre),2022-07-12 05:01:38,2022-07-12 05:06:14,2022-07-12 05:12:31,10.883333,4.600000,6.283333
...,...,...,...,...,...,...,...,...,...
200010,50.61287,5.84386,verviers (verviers),2023-01-02 21:36:05,2023-01-02 21:41:12,2023-01-02 21:46:52,10.783333,5.116667,5.666667
200209,50.50642,5.66226,aywaille (sougn√©-remouchamps),2023-01-03 14:56:03,2023-01-03 14:59:33,2023-01-03 15:04:47,8.733333,3.500000,5.233333
200210,50.50642,5.66226,li√®ge (angleur),2023-01-03 14:56:03,2023-01-03 15:00:24,2023-01-03 15:13:15,17.200000,4.350000,12.850000
200477,50.72596,5.34973,4342 awans (hognoul),2023-01-04 13:43:56,2023-01-04 13:47:07,2023-01-04 13:56:09,12.216667,3.183333,9.033333


In [5]:
interventions3 = pd.read_parquet("interventions3.parquet") 

interventions3_car = interventions3[interventions3['EventType Trip'] == 'P003 - Cardiac arrest']
interventions3_car = interventions3_car[['Latitude intervention', 'Longitude intervention', 'CityName permanence', 'T0', 'T2', 'T3']]
interventions3_car['T2'] = pd.to_datetime(interventions3_car['T2'], format="%Y-%m-%d %H:%M:%S.%f").dt.floor('S')
interventions3_car['T3'] = pd.to_datetime(interventions3_car['T3'], format="%Y-%m-%d %H:%M:%S.%f").dt.floor('S')
interventions3_car['T0'] = pd.to_datetime(interventions3_car['T0'], format="%d%b%y:%H:%M:%S").dt.floor('S')

interventions3_car['T3-T0'] = (interventions3_car['T3'] - interventions3_car['T0']).dt.total_seconds() / 60
interventions3_car['T2-T0'] = (interventions3_car['T2'] - interventions3_car['T0']).dt.total_seconds() / 60
interventions3_car['T3-T2'] = (interventions3_car['T3'] - interventions3_car['T2']).dt.total_seconds() / 60


interventions3_car.rename(columns={'Latitude intervention': 'Lat', 'Longitude intervention': 'Lon', 'CityName permanence': 'City'}, inplace=True)
interventions3_car['City'] = interventions3_car['City'].str.lower()

interventions3_car


,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
27,50.73437,5.67101,oupeye (oupeye),2023-01-05 06:44:32,2023-01-05 06:49:28,2023-01-05 06:53:16,8.733333,4.933333,3.800000
150,50.74748,2.93860,heuvelland (nieuwkerke),2023-01-05 13:35:10,2023-01-05 13:44:23,2023-01-05 13:54:13,19.050000,9.216667,9.833333
151,50.74748,2.93860,wervik (wervik),2023-01-05 13:35:10,NaT,NaT,NaN,NaN,NaN
152,50.74748,2.93860,ieper (sint-jan),2023-01-05 13:35:10,2023-01-05 13:40:00,2023-01-05 13:52:22,17.200000,4.833333,12.366667
176,50.60025,5.53550,seraing (ougr√©e),2023-01-05 14:39:31,2023-01-05 14:42:22,2023-01-05 15:34:56,55.416667,2.850000,52.566667
...,...,...,...,...,...,...,...,...,...
200114,NaN,489.93400,eghezèe (eghezèe),2023-05-29 03:04:56,NaT,NaT,NaN,NaN,NaN
200235,NaN,557.72400,liëge (liëge),2023-05-29 20:28:42,NaT,NaT,NaN,NaN,NaN
200474,NaN,490.60700,eghezèe (eghezèe),2023-05-31 06:17:38,NaT,NaT,NaN,NaN,NaN
200475,NaN,489.26400,namur (saint-servais),2023-05-31 06:45:01,NaT,NaT,NaN,NaN,NaN


In [6]:
interventions_bxl = pd.read_parquet("interventions_bxl.parquet.gzip")

interventions_bxl_car = interventions_bxl[interventions_bxl['eventtype_trip'] == 'P003 - Cardiac arrest']

interventions_bxl_car = interventions_bxl_car[['latitude_intervention', 'longitude_intervention', 'cityname_permanence', 't0', 't2', 't3']]

interventions_bxl_car.rename(columns={'t0': 'T0', 't2': 'T2', 't3': 'T3'}, inplace=True)


import pandas as pd

# convert the date to datetime
interventions_bxl_car['T2'] = pd.to_datetime(interventions_bxl_car['T2'], utc=True).dt.tz_convert('Europe/Brussels')
interventions_bxl_car['T3'] = pd.to_datetime(interventions_bxl_car['T3'], utc=True).dt.tz_convert('Europe/Brussels')
interventions_bxl_car['T0'] = pd.to_datetime(interventions_bxl_car['T0'], utc=True).dt.tz_convert('Europe/Brussels')

# # delete the timezone information
interventions_bxl_car['T2'] = interventions_bxl_car['T2'].dt.tz_localize(None).dt.floor('S')
interventions_bxl_car['T3'] = interventions_bxl_car['T3'].dt.tz_localize(None).dt.floor('S')
interventions_bxl_car['T0'] = interventions_bxl_car['T0'].dt.tz_localize(None).dt.floor('S')



# calculate the time difference with minutes
interventions_bxl_car['T3-T0'] = (interventions_bxl_car['T3'] - interventions_bxl_car['T0']).dt.total_seconds() / 60
interventions_bxl_car['T2-T0'] = (interventions_bxl_car['T2'] - interventions_bxl_car['T0']).dt.total_seconds() / 60
interventions_bxl_car['T3-T2'] = (interventions_bxl_car['T3'] - interventions_bxl_car['T2']).dt.total_seconds() / 60

interventions_bxl_car.rename(columns={'latitude_intervention': 'Lat', 'longitude_intervention': 'Lon','cityname_permanence': 'City'}, inplace=True)

# Convert the 'Lat' and 'Lon' column values to strings
interventions_bxl_car['Lat_str'] = interventions_bxl_car['Lat'].astype(int).astype(str)
interventions_bxl_car['Lon_str'] = interventions_bxl_car['Lon'].astype(int).astype(str)

# Apply string manipulation to insert a decimal point after the first two digits
interventions_bxl_car['Lat_new'] = interventions_bxl_car['Lat_str'].apply(lambda x: x[:2] + '.' + x[2:])
interventions_bxl_car['Lon_new'] = interventions_bxl_car['Lon_str'].apply(lambda x: x[:1] + '.' + x[1:])

# Convert the manipulated strings back to floats
interventions_bxl_car['Lat'] = interventions_bxl_car['Lat_new'].astype(float)
interventions_bxl_car['Lon'] = interventions_bxl_car['Lon_new'].astype(float)

# Drop the temporary columns used for the transformation
interventions_bxl_car.drop(['Lat_str', 'Lat_new', 'Lon_str', 'Lon_new'], axis=1, inplace=True)




interventions_bxl_car['City'] = interventions_bxl_car['City'].str.lower()
interventions_bxl_car




,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
205,50.83808,4.30484,anderlecht,2022-09-06 20:31:42,NaT,NaT,NaN,NaN,NaN
206,50.83808,4.30484,anderlecht (anderlecht),2022-09-06 20:31:42,2022-09-06 20:38:18,2022-09-06 20:44:39,12.950000,6.600000,6.350000
207,50.83808,4.30484,anderlecht,2022-09-06 20:31:42,2022-09-06 20:38:46,NaT,NaN,7.066667,NaN
316,50.82480,4.38185,elsene (elsene),2022-09-07 02:36:15,2022-09-07 02:41:59,2022-09-07 02:42:25,6.166667,5.733333,0.433333
317,50.82480,4.38185,elsene,2022-09-07 02:36:15,2022-09-07 02:41:36,2022-09-07 02:44:59,8.733333,5.350000,3.383333
...,...,...,...,...,...,...,...,...,...
114339,50.86529,4.39050,oudergem,2023-05-28 21:11:05,2023-05-28 21:16:55,NaT,NaN,5.833333,NaN
114554,50.83360,4.34023,bruxelles (bruxelles),2023-05-29 12:36:13,2023-05-29 12:41:16,2023-05-29 12:44:07,7.900000,5.050000,2.850000
114555,50.83360,4.34023,brussel (brussel),2023-05-29 12:36:13,2023-05-29 12:41:10,2023-05-29 12:44:02,7.816667,4.950000,2.866667
115025,50.85911,4.28499,jette (jette),2023-05-30 15:42:36,2023-05-30 15:45:27,2023-05-30 15:57:32,14.933333,2.850000,12.083333


In [7]:
interventions_bxl2 = pd.read_parquet("interventions_bxl2.parquet.gzip")

pattern = "P003"
interventions_bxl2_car = interventions_bxl2[interventions_bxl2['EventType and EventLevel'].str.contains(pattern, na=False, regex=False)]


interventions_bxl2_car = interventions_bxl2_car[['Latitude intervention', 'Longitude intervention', 'Cityname Intervention', 'T0', 'T2', 'T3']]
interventions_bxl2_car['T2'] = pd.to_datetime(interventions_bxl2_car['T2'], format="%d%b%y:%H:%M:%S").dt.floor('S')
interventions_bxl2_car['T3'] = pd.to_datetime(interventions_bxl2_car['T3'], format="%d%b%y:%H:%M:%S").dt.floor('S')
interventions_bxl2_car['T0'] = pd.to_datetime(interventions_bxl2_car['T0'], format="%d%b%y:%H:%M:%S").dt.floor('S')

interventions_bxl2_car['T3-T0'] = (interventions_bxl2_car['T3'] - interventions_bxl2_car['T0']).dt.total_seconds() / 60
interventions_bxl2_car['T2-T0'] = (interventions_bxl2_car['T2'] - interventions_bxl2_car['T0']).dt.total_seconds() / 60
interventions_bxl2_car['T3-T2'] = (interventions_bxl2_car['T3'] - interventions_bxl2_car['T2']).dt.total_seconds() / 60

interventions_bxl2_car.rename(columns={'Latitude intervention': 'Lat', 'Longitude intervention': 'Lon', 'Cityname Intervention':'City'}, inplace=True)

# Convert the 'Lat' and 'Lon' column values to strings
interventions_bxl2_car['Lat_str'] = interventions_bxl2_car['Lat'].astype(int).astype(str)
interventions_bxl2_car['Lon_str'] = interventions_bxl2_car['Lon'].astype(int).astype(str)

# Apply string manipulation to insert a decimal point after the first two digits
interventions_bxl2_car['Lat_new'] = interventions_bxl2_car['Lat_str'].apply(lambda x: x[:2] + '.' + x[2:])
interventions_bxl2_car['Lon_new'] = interventions_bxl2_car['Lon_str'].apply(lambda x: x[:1] + '.' + x[1:])

# Convert the manipulated strings back to floats
interventions_bxl2_car['Lat'] = interventions_bxl2_car['Lat_new'].astype(float)
interventions_bxl2_car['Lon'] = interventions_bxl2_car['Lon_new'].astype(float)

# Drop the temporary columns used for the transformation
interventions_bxl2_car.drop(['Lat_str', 'Lat_new', 'Lon_str', 'Lon_new'], axis=1, inplace=True)


interventions_bxl2_car['City'] = interventions_bxl2_car['City'].apply(lambda x: x[5:]).str.lower()
interventions_bxl2_car

,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
808,50.849492,4.398207,schaerbeek (schaerbeek),2022-06-02 20:11:21,2022-06-02 20:16:53,NaT,NaN,5.533333,NaN
809,50.849492,4.398207,schaerbeek (schaerbeek),2022-06-02 20:11:21,NaT,NaT,NaN,NaN,NaN
1074,50.843201,4.343397,bruxelles (bruxelles),2022-06-03 11:46:26,2022-06-03 11:50:28,2022-06-03 11:54:26,8.000000,4.033333,3.966667
1075,50.843201,4.343397,bruxelles (bruxelles),2022-06-03 11:46:26,NaT,NaT,NaN,NaN,NaN
1622,50.881461,4.347456,bruxelles (laeken),2022-06-04 16:11:01,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36258,50.841062,4.389640,brussel (brussel),2022-08-31 17:55:56,2022-08-31 18:01:05,2022-08-31 18:06:07,10.183333,5.150000,5.033333
36414,50.844022,4.362110,bruxelles (bruxelles),2022-09-01 08:35:12,2022-09-01 08:40:13,2022-09-01 08:44:53,9.683333,5.016667,4.666667
36415,50.844022,4.362110,bruxelles (bruxelles),2022-09-01 08:35:12,2022-09-01 08:39:32,2022-09-01 08:42:56,7.733333,4.333333,3.400000
38298,50.901283,4.348130,bruxelles (laeken),2022-09-05 17:45:18,NaT,NaT,NaN,NaN,NaN


In [8]:

cad9 = pd.read_parquet("cad9.parquet.gzip") # cad

cad9_car = cad9[cad9['EventType Trip'] == 'P003 - HARTSTILSTAND - DOOD - OVERLEDEN']
cad9_car = cad9_car[['Latitude intervention', 'Longitude intervention', 'CityName intervention', 'T0', 'T2', 'T3']]
cad9_car['T2'] = pd.to_datetime(cad9_car['T2']).dt.floor('S')
cad9_car['T3'] = pd.to_datetime(cad9_car['T3']).dt.floor('S')
cad9_car['T0'] = pd.to_datetime(cad9_car['T0']).dt.floor('S')

cad9_car['T3-T0'] = (cad9_car['T3'] - cad9_car['T0']).dt.total_seconds() / 60
cad9_car['T2-T0'] = (cad9_car['T2'] - cad9_car['T0']).dt.total_seconds() / 60
cad9_car['T3-T2'] = (cad9_car['T3'] - cad9_car['T2']).dt.total_seconds() / 60

cad9_car.rename(columns={'Latitude intervention': 'Lat', 'Longitude intervention': 'Lon', 'CityName intervention': 'City'}, inplace=True)

cad9_car['City'] = cad9_car['City'].str.lower()

cad9_car




,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
322,50.928019,4.535123,kampenhout,2022-07-15 15:21:57,2022-06-02 14:11:32,2022-06-02 14:11:34,-61990.383333,-61990.416667,0.033333
323,50.928019,4.535123,kampenhout,2022-07-15 15:21:57,2022-06-02 14:11:32,2022-06-02 14:11:34,-61990.383333,-61990.416667,0.033333
440,50.767029,4.273000,beersel,2022-06-02 23:04:09,2022-06-02 23:07:38,2022-06-02 23:13:59,9.833333,3.483333,6.350000
441,50.767029,4.273000,beersel,2022-06-02 23:04:09,2022-06-02 23:07:38,2022-06-02 23:13:59,9.833333,3.483333,6.350000
516,50.898349,4.755029,lubbeek,2022-06-03 09:43:11,2022-06-03 09:49:08,2022-06-03 09:58:14,15.050000,5.950000,9.100000
...,...,...,...,...,...,...,...,...,...
289012,50.841218,3.902211,herzele,2023-05-30 19:05:35,2023-05-30 19:09:42,2023-05-30 19:19:42,14.116667,4.116667,10.000000
289135,51.127259,3.676424,evergem,2023-05-31 07:04:47,2023-05-31 07:09:10,2023-05-31 07:21:31,16.733333,4.383333,12.350000
289136,51.127259,3.676424,evergem,2023-05-31 07:04:47,2023-05-31 07:09:10,2023-05-31 07:21:31,16.733333,4.383333,12.350000
289355,50.896206,3.856032,herzele,2023-05-31 20:45:48,2023-05-31 20:53:01,2023-05-31 20:57:17,11.483333,7.216667,4.266667


In [9]:
# Merge the dataframes
frames = [interventions1_car, interventions2_car, interventions3_car, interventions_bxl_car, interventions_bxl2_car, cad9_car]

result = pd.concat(frames).reset_index(drop=True)



result

,Lat,Lon,City,T0,T2,T3,T3-T0,T2-T0,T3-T2
0,51.206850,5.136620,geel (geel),2022-06-01 06:45:53,2022-06-01 06:52:06,2022-06-01 07:04:46,18.883333,6.216667,12.666667
1,51.248480,4.438690,antwerpen (albertdok),2022-06-01 09:28:15,2022-06-01 09:30:28,2022-06-01 09:32:29,4.233333,2.216667,2.016667
2,51.248480,4.438690,antwerpen (antwerpen),2022-06-01 09:28:15,2022-06-01 09:35:10,2022-06-01 09:40:51,12.600000,6.916667,5.683333
3,51.282730,4.421340,antwerpen (antwerpen),2022-06-01 15:11:15,2022-06-01 15:14:47,2022-06-01 15:24:40,13.416667,3.533333,9.883333
4,51.282730,4.421340,antwerpen (merksem),2022-06-01 15:11:15,2022-06-01 15:17:56,2022-06-01 15:47:52,36.616667,6.683333,29.933333
...,...,...,...,...,...,...,...,...,...
13564,50.841218,3.902211,herzele,2023-05-30 19:05:35,2023-05-30 19:09:42,2023-05-30 19:19:42,14.116667,4.116667,10.000000
13565,51.127259,3.676424,evergem,2023-05-31 07:04:47,2023-05-31 07:09:10,2023-05-31 07:21:31,16.733333,4.383333,12.350000
13566,51.127259,3.676424,evergem,2023-05-31 07:04:47,2023-05-31 07:09:10,2023-05-31 07:21:31,16.733333,4.383333,12.350000
13567,50.896206,3.856032,herzele,2023-05-31 20:45:48,2023-05-31 20:53:01,2023-05-31 20:57:17,11.483333,7.216667,4.266667


In [10]:
# Save the dataframe to csv
result.to_csv("cardiac_arrest.csv", index=False)

In [11]:
result_leuven = result[((result['City'] == 'leuven') | (result['City'] == 'leuven (heverlee)'))]
result_leuven.to_csv("cardiac_arrest_leuven.csv", index=False)
